In [1]:
from pyspark.sql import SparkSession

from graphframes import GraphFrame

session = SparkSession.builder.master('local').getOrCreate()

21/09/14 00:21:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
used_df = session.read.format("csv").option("header", "true").load("user.csv") \
                .withColumnRenamed("ID", "id")

used_df.show(5)
used_df = used_df.sample(0.2)

+----+------+
|  id|  NAME|
+----+------+
|1090|Jessie|
|1159|Melvin|
|1436| Hosea|
|1512| Ernst|
|1572|Finley|
+----+------+
only showing top 5 rows



In [3]:
edges_df = session.read.format("csv").option("header", "true").load("edges.csv") \
                .withColumnRenamed("USER_1", "src") \
                .withColumnRenamed("USER_2", "dst")

edges_df.show(5)
edges_df = edges_df.sample(0.2)

+----+----+
| src| dst|
+----+----+
|1090|5309|
|1090|3547|
|1090|5166|
|1090|5307|
|1159|1807|
+----+----+
only showing top 5 rows



In [4]:
g = GraphFrame(used_df, edges_df)


In [5]:
g.vertices.count()

1281

In [6]:
g.edges.count()

66874

In [11]:
g.find("(a)-[e1]->(b)") \
.filter("a.id = 5307") \
.select("b.id", "b.NAME") \
.show()

+----+--------+
|  id|    NAME|
+----+--------+
|4704|   Ammon|
|2664|    Edna|
|3953|Gottlieb|
| 829|    Cato|
+----+--------+



In [8]:
g.find("(a)-[e1]->(b); (b)-[e2]->(c); !(a)-[]->(c)") \
.filter("a.id = 1090") \
.select("c.id", "c.NAME") \
.show()

+----+--------+
|  id|    NAME|
+----+--------+
|2664|    Edna|
|3953|Gottlieb|
|4704|   Ammon|
| 829|    Cato|
+----+--------+



In [13]:
g.inDegrees.show()

+----+--------+
|  id|inDegree|
+----+--------+
|3210|      29|
|2069|      53|
|3959|      26|
|4937|       3|
|3606|      28|
|5925|      28|
|5325|       4|
|1572|       8|
|2088|      10|
|2904|       8|
|6194|       4|
|1159|       2|
| 691|       3|
| 829|      11|
| 675|       1|
|6240|       2|
|4032|       2|
|1512|       1|
|2294|       3|
|2162|       8|
+----+--------+
only showing top 20 rows

